In [16]:
from transformers import BertTokenizer, BertModel

import torch
from torch import nn

from dataset import Dataset

import sys
import numpy as np
import pickle

np.set_printoptions(threshold=sys.maxsize)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
with open('train_data_1500.pickle', 'rb') as train_data_file:
    data = pickle.load(train_data_file)
    
print(data['1401-0']['question'][0][7])

[-3.91157925e-01  2.02038288e-01 -6.00516610e-02 -9.87990737e-01
  1.47863030e-01  1.65864989e-01  7.44144320e-01  2.84907222e-01
 -1.09637588e-01 -1.12793006e-01 -1.77599102e-01 -3.68774146e-01
 -3.39690000e-01 -2.86659896e-01  5.33435524e-01  6.42223895e-01
  5.52588761e-01 -3.38813007e-01 -6.50696456e-02  1.23105454e+00
  7.60272264e-01  7.29751825e-01 -2.04614718e-02 -1.30072132e-01
  3.57189596e-01  5.92365339e-02  6.73542798e-01 -6.56539574e-03
 -5.07208966e-02 -4.77240741e-01  5.64617693e-01  4.10353206e-02
 -2.15326756e-01 -1.61748528e-01  2.87019044e-01 -4.30164874e-01
  1.49317756e-01 -4.80662465e-01  4.22185585e-02  9.48468745e-01
 -6.97192430e-01 -7.60580182e-01  3.03396702e-01 -3.29173803e-01
 -6.04220629e-01 -1.12431300e+00 -3.63771826e-01 -5.19177914e-01
  8.56252611e-01  1.47997260e-01 -2.05536690e-02  7.08647370e-01
  2.63067186e-02  2.79360473e-01 -7.55071104e-01  1.24056661e+00
  7.94229865e-01 -6.74854875e-01 -1.34086728e-01 -5.64876914e-01
  3.61617953e-01 -3.88168

In [29]:
class BertBinaryClassifier(nn.Module):
    def __init__(self):
        super(BertBinaryClassifier, self).__init__()
        self.dropout1 = nn.Dropout(0.1)
        self.linear = nn.Linear(768, 90)
        # self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens, masks=None):
        
        # _, pooled_output = self.bert(tokens, attention_mask=masks)
        # dropout_output = self.dropout(pooled_output)
        dropout_output = self.dropout1(tokens)
        linear_output = self.linear(dropout_output)
        return linear_output

In [10]:
datasets = []

file_paths_train = glob.glob('../data/train_data_pre_processed/*.pkl')

for file_path_train in file_paths_train:
    datasets.append(Dataset(file_path_train))

dataset = ConcatDataset(datasets)
params = {'batch_size': 1,
          'shuffle': True,
          'num_workers': 1}

training_generator = torch.utils.data.DataLoader(dataset, **params)

In [27]:
def predict_class(model, X):
    model = model.eval()
    outputs = model(X)
    _, pred = torch.max(outputs.data, 1)
    model = model.train()
    return pred.numpy()

def predict(model, X, T=10):
    standard_pred = predict_class(model, X)
    y1 = []
    y2 = []
    for _ in range(T):
        _y1 = model(X)
        _y2 = F.softmax(_y1, dim=1)
        y1.append(_y1.data.numpy())
        y2.append(_y2.data.numpy())
    return standard_pred, np.array(y1), np.array(y2)

In [28]:
model = BertBinaryClassifier()
X1, y1 = iter(training_generator).next()

# Need to flatten X1 before feeding into MLP
y1_pred, y1_si, y1_so = predict(model, X1)
print('Predictions: {}'.format(y1_pred))

asd
Predictions: [[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 